# Image Folder to PDF Converter

Convert all images in a folder to a single, optimized, portrait-oriented PDF. Fast, compact, and compatible with all PDF viewers.

**Steps:**
1. Run the setup cell below.
2. Edit the folder and output path in the main cell.
3. Run the main cell to create your PDF.


In [ ]:
# Package Installation

!python3 -m venv PDF --clear
!source PDF/bin/activate && pip install --upgrade pip
!source PDF/bin/activate && pip install PyMuPDF Pillow tqdm

print("Setup complete! Ready to convert images to PDF.")

  Using cached pip-25.1.1-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-25.1.1-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0


In [ ]:
import os
import io
from PIL import Image
from tqdm import tqdm
import multiprocessing
import fitz  # PyMuPDF

print("Image Folder to PDF Converter)")
print("=" * 60)

# --- USER INPUT SECTION ---
image_folder = "/home/bacon/PDF/Photos/JPG"  
output_pdf_path = "/home/bacon/PDF/Biomortem_dwn.pdf"  

# Optimized settings for portrait PDFs
max_width = 1200   # Maximum width in pixels (portrait orientation)
max_height = 1600  # Maximum height in pixels (taller for portrait)
jpeg_quality = 85  # JPEG quality (0-100)
force_portrait = True  # Force all images to portrait orientation

# Rotation settings (if images are upside down, try different values)
rotation_angle = -90  # Degrees to rotate landscape images (-90, 90, 180, 270)
                      # -90 = counter-clockwise, 90 = clockwise
                      # Try 90 if images are upside down with -90
# --- END USER INPUT SECTION ---

print(f"Source folder: {image_folder}")
print(f"Output PDF: {output_pdf_path}")
print(f"Portrait mode: {'Enabled' if force_portrait else 'Disabled'}")
if force_portrait:
    print(f"🔄 Rotation angle: {rotation_angle}° ({'Counter-clockwise' if rotation_angle < 0 else 'Clockwise'})")
print()

# Get sorted list of image files
image_files = sorted([
    os.path.join(image_folder, f)
    for f in os.listdir(image_folder)
    if f.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tiff"))
])

if not image_files:
    print("❌ No image files found in the specified folder!")
    exit()

print(f"🔍 Found {len(image_files)} images")

# CPU configuration
num_procs = os.cpu_count() or 1
print(f"🚀 Using {num_procs} CPU cores for multiprocessing")
print()

# Optimized image processing function with smart rotation
def process_image_final(args):
    idx, img_path, max_width, max_height, jpeg_quality, force_portrait, rotation_angle = args
    
    try:
        with Image.open(img_path) as img:
            # Convert to RGB if needed
            if img.mode != "RGB":
                img = img.convert("RGB")
            
            # Handle EXIF orientation data to fix camera rotation issues
            try:
                # Use the built-in method to handle EXIF orientation
                img = img._getexif() and img._getexif().get(274, 1) and img or img
                # Apply proper EXIF orientation
                if hasattr(img, '_getexif') and img._getexif():
                    exif = img._getexif()
                    orientation = exif.get(274, 1)  # Orientation tag
                    
                    if orientation == 3:
                        img = img.rotate(180, expand=True)
                    elif orientation == 6:
                        img = img.rotate(270, expand=True)
                    elif orientation == 8:
                        img = img.rotate(90, expand=True)
            except (AttributeError, KeyError, TypeError):
                pass
            
            # Apply transpose method to automatically fix orientation
            exif_corrected = False
            try:
                from PIL.ExifTags import ORIENTATION
                if hasattr(img, 'getexif'):
                    exif_dict = img.getexif()
                    if ORIENTATION in exif_dict:
                        orientation = exif_dict[ORIENTATION]
                        if orientation == 3:
                            img = img.transpose(Image.ROTATE_180)
                            exif_corrected = True
                        elif orientation == 6:
                            img = img.transpose(Image.ROTATE_270)
                            exif_corrected = True  
                        elif orientation == 8:
                            img = img.transpose(Image.ROTATE_90)
                            exif_corrected = True
            except:
                pass
            
            # Force portrait orientation if requested
            rotated = False
            applied_rotation_angle = 0
            if force_portrait and img.width > img.height:
                # Use the user-specified rotation angle
                img = img.rotate(rotation_angle, expand=True)
                applied_rotation_angle = rotation_angle
                rotated = True
            
            # Calculate new size while maintaining aspect ratio (ONLY DOWNSCALE)
            original_width, original_height = img.size
            aspect_ratio = original_width / original_height
            
            # Only resize if the image is larger than our max dimensions
            resized = False
            if original_width > max_width or original_height > max_height:
                if aspect_ratio > 1:  # Landscape (shouldn't happen after rotation)
                    new_width = min(max_width, original_width)
                    new_height = int(new_width / aspect_ratio)
                else:  # Portrait
                    new_height = min(max_height, original_height)
                    new_width = int(new_height * aspect_ratio)
                
                # Only resize if we're making it smaller (never upscale)
                if new_width < original_width or new_height < original_height:
                    img = img.resize((new_width, new_height), Image.LANCZOS)
                    resized = True
            
            # Save as JPEG to bytes
            img_bytes_io = io.BytesIO()
            img.save(img_bytes_io, format="JPEG", quality=jpeg_quality)
            img_bytes = img_bytes_io.getvalue()
            width, height = img.size
            
            return (idx, img_bytes, width, height, rotated, resized, applied_rotation_angle, exif_corrected)
            
    except Exception as e:
        print(f"⚠️  Error processing {os.path.basename(img_path)}: {e}")
        return None

# Prepare arguments for multiprocessing
args_list = [(i, img_path, max_width, max_height, jpeg_quality, force_portrait, rotation_angle) 
             for i, img_path in enumerate(image_files)]

print("🔄 Processing images...")
results = []
rotation_count = 0
resize_count = 0
exif_correction_count = 0
rotation_angles = []

with multiprocessing.Pool(processes=num_procs) as pool:
    for result in tqdm(pool.imap_unordered(process_image_final, args_list), 
                      total=len(args_list), desc="Processing"):
        if result:
            results.append(result)
            if len(result) >= 7 and result[4]:  # rotated (handle both old and new format)
                rotation_count += 1
                if len(result) >= 7:
                    rotation_angles.append(result[6])  # rotation angle
            if len(result) >= 6 and result[5]:  # resized
                resize_count += 1
            if len(result) >= 8 and result[7]:  # exif corrected
                exif_correction_count += 1

# Sort results by original order
results.sort(key=lambda x: x[0])

print(f"✅ Processed {len(results)} images successfully")
print(f"🔧 Applied EXIF orientation correction to {exif_correction_count} images")
print(f"🔄 Rotated {rotation_count} landscape images to portrait")
if rotation_angles:
    print(f"   └─ Rotation angles used: {set(rotation_angles)} degrees")
print(f"📏 Resized {resize_count} images for optimization")
print()

if not results:
    print("❌ No images were processed successfully!")
    exit()

# Create PDF
print("📚 Assembling PDF...")
new_doc = fitz.open()

for idx, result in enumerate(tqdm(results, desc="Writing pages")):
    try:
        # Handle both old and new result formats
        if len(result) >= 8:
            i, img_bytes, width, height, rotated, resized, rotation_angle, exif_corrected = result
        elif len(result) >= 7:
            i, img_bytes, width, height, rotated, resized, rotation_angle = result
        else:
            i, img_bytes, width, height, rotated, resized = result
        
        img_stream = io.BytesIO(img_bytes)
        new_page_rect = fitz.Rect(0, 0, width, height)
        new_page = new_doc.new_page(width=width, height=height)
        new_page.insert_image(new_page_rect, stream=img_stream)
        
        # Progress update every 50 pages
        if (idx + 1) % 50 == 0 or (idx + 1) == len(results):
            print(f"  📄 Written {idx + 1}/{len(results)} pages...")
            
    except Exception as e:
        print(f"⚠️  Error adding page {idx + 1}: {e}")

# Save PDF
try:
    new_doc.save(output_pdf_path)
    new_doc.close()
    print(f"PDF saved successfully!")
except Exception as e:
    print(f"❌ Error saving PDF: {e}")
    exit()

# Final validation and statistics
print()
print("🔍 Final validation...")

try:
    # Validate PDF
    doc_final = fitz.open(output_pdf_path)
    page_count = doc_final.page_count
    
    # Check orientations
    portrait_pages = 0
    landscape_pages = 0
    for i in range(page_count):
        page = doc_final[i]
        if page.rect.height > page.rect.width:
            portrait_pages += 1
        else:
            landscape_pages += 1
    
    # Get file size
    file_size_mb = os.path.getsize(output_pdf_path) / (1024 * 1024)
    avg_size_per_page = file_size_mb / page_count if page_count > 0 else 0
    
    doc_final.close()
    
    # Success report
    print("=" * 60)
    print("SUCCESS! PDF Created Successfully")
    print("=" * 60)
    print(f"Output file: {output_pdf_path}")
    print(f"Total pages: {page_count}")
    print(f"Portrait pages: {portrait_pages} ({portrait_pages/page_count*100:.1f}%)")
    print(f"Landscape pages: {landscape_pages} ({landscape_pages/page_count*100:.1f}%)")
    print(f"File size: {file_size_mb:.1f} MB")
    print(f"Average per page: {avg_size_per_page:.2f} MB")
    print()
    
    if force_portrait and portrait_pages == page_count:
        print("🎉 Perfect! All images are now in portrait orientation!")
        print("🔧 Applied smart rotation with EXIF orientation correction")
    elif force_portrait:
        print(f"⚠️  Note: {landscape_pages} images remained in landscape orientation")
    
except Exception as e:
    print(f"⚠️  Validation error (but PDF was created): {e}")
    print(f"📄 Check the file manually: {output_pdf_path}")

print("🏁 Process completed!")

Image Folder to PDF Converter (Portrait Orientation)
Source folder: /home/bacon/PDF/Photos/JPG
Output PDF: /home/bacon/PDF/Biomortem_dwn.pdf
Portrait mode: Enabled
🔄 Rotation angle: -90° (Counter-clockwise)

🔍 Found 252 images
🚀 Using 28 CPU cores for multiprocessing

🔄 Processing images...


Processing: 100%|██████████| 252/252 [00:07<00:00, 33.40it/s]



✅ Processed 252 images successfully
🔧 Applied EXIF orientation correction to 0 images
🔄 Rotated 0 landscape images to portrait
📏 Resized 252 images for optimization

📚 Assembling PDF...


Writing pages:  18%|█▊        | 45/252 [00:00<00:00, 441.72it/s]

  📄 Written 50/252 pages...


...

  📄 Written 100/252 pages...
  📄 Written 150/252 pages...


Writing pages:  76%|███████▌  | 191/252 [00:00<00:00, 471.97it/s]

  📄 Written 200/252 pages...


Writing pages: 100%|██████████| 252/252 [00:00<00:00, 468.83it/s]


  📄 Written 250/252 pages...
  📄 Written 252/252 pages...
PDF saved successfully!

🔍 Final validation...
SUCCESS! PDF Created Successfully
Output file: /home/bacon/PDF/Biomortem_dwn.pdf
Total pages: 252
Portrait pages: 252 (100.0%)
Landscape pages: 0 (0.0%)
File size: 106.9 MB
Average per page: 0.42 MB

🎉 Perfect! All images are now in portrait orientation!
🔧 Applied smart rotation with EXIF orientation correction
🏁 Process completed!
SUCCESS! PDF Created Successfully
Output file: /home/bacon/PDF/Biomortem_dwn.pdf
Total pages: 252
Portrait pages: 252 (100.0%)
Landscape pages: 0 (0.0%)
File size: 106.9 MB
Average per page: 0.42 MB

🎉 Perfect! All images are now in portrait orientation!
🔧 Applied smart rotation with EXIF orientation correction
🏁 Process completed!
